# Lab 4 Multi-variable linear regression

## Lab04-4-tf_reader_linear_regression

### 아주 많은 데이터(파일이 여러개로 나눠있는 등)를 다루는 방법
> https://www.tensorflow.org/programmers_guide/reading_data

In [1]:
import tensorflow as tf

In [2]:
tf.set_random_seed(777)  # for reproducibility

In [3]:
# 파일 리스트를 넣어줌(여기서는 1개만 함)
# ex) ['data-01-test-score.csv', 'data-02-test-score.csv', ...]
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')   # shuffle=True : 파일 순서가 랜덤하게 shuffle됨

In [4]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [5]:
# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]  # decode시 형태???
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [6]:
# collect batches of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [7]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [8]:
W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [9]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

In [10]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [11]:
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [12]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [13]:
# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [14]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  7070.98 
Prediction:
 [[ 235.22784424]
 [ 282.40145874]
 [ 278.3961792 ]
 [ 303.91577148]
 [ 214.6239624 ]
 [ 159.1542511 ]
 [ 228.53509521]
 [ 170.92643738]
 [ 264.65048218]
 [ 246.59115601]]
10 Cost:  5.57779 
Prediction:
 [[ 154.65847778]
 [ 185.55499268]
 [ 182.97673035]
 [ 199.99612427]
 [ 140.76091003]
 [ 104.69593811]
 [ 150.7492981 ]
 [ 113.26345825]
 [ 174.13757324]
 [ 162.75192261]]
20 Cost:  4.62937 
Prediction:
 [[ 154.11100769]
 [ 184.90293884]
 [ 182.33100891]
 [ 199.2953949 ]
 [ 140.26301575]
 [ 104.33343506]
 [ 150.22685242]
 [ 112.87885284]
 [ 173.53265381]
 [ 162.19770813]]
30 Cost:  4.61896 
Prediction:
 [[ 154.10362244]
 [ 184.90007019]
 [ 182.32485962]
 [ 199.29121399]
 [ 140.26023865]
 [ 104.33596039]
 [ 150.2257843 ]
 [ 112.88076019]
 [ 173.53443909]
 [ 162.20545959]]
40 Cost:  4.61285 
Prediction:
 [[ 154.09985352]
 [ 184.90153503]
 [ 182.32302856]
 [ 199.29173279]
 [ 140.26077271]
 [ 104.34094238]
 [ 150.22822571]
 [ 112.88526917]
 [ 173.54026794]
 [ 1

In [15]:
coord.request_stop()
coord.join(threads)

In [16]:
# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Your score will be  [[ 185.33529663]]
Other scores will be  [[ 178.36245728]
 [ 177.03686523]]
